In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Datos de ejemplo para entrenar el chatbot (preguntas y respuestas)
data = [
    ("hola", "¡Hola! ¿Cómo estás?"),
    ("¿cómo estás?", "Estoy bien, gracias por preguntar."),
    ("adiós", "¡Adiós! Cuídate."),
    ("¿qué es tu nombre?", "Soy un chatbot simple."),
]

# Preprocesamiento del texto
def tokenize(sentence):
    return sentence.lower().split()

def build_vocab(data):
    words = set()
    for question, answer in data:
        words.update(tokenize(question))
    return {word: idx for idx, word in enumerate(words)}

vocab = build_vocab(data)
vocab_size = len(vocab)
print("Vocabulario:", vocab)

# Convertir texto a tensores
def text_to_tensor(text, vocab):
    tokens = tokenize(text)
    indices = [vocab[word] for word in tokens if word in vocab]
    return torch.tensor(indices, dtype=torch.long)

# Crear una red neuronal simple
class ChatbotModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes):
        super(ChatbotModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        _, (h_n, _) = self.lstm(x)
        out = self.fc(h_n[-1])
        return out

# Hiperparámetros
embed_size = 8
hidden_size = 16
num_classes = len(data)
model = ChatbotModel(vocab_size, embed_size, hidden_size, num_classes)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Entrenamiento del modelo
epochs = 500
for epoch in range(epochs):
    total_loss = 0
    for idx, (question, answer) in enumerate(data):
        x = text_to_tensor(question, vocab).unsqueeze(0)  # Agregar dimensión batch
        y = torch.tensor([idx])  # Índice de la respuesta correcta
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(data):.4f}")

# Guardar el modelo en formato TorchScript
example_input = text_to_tensor("hola", vocab).unsqueeze(0)
scripted_model = torch.jit.trace(model, example_input)
torch.jit.save(scripted_model, 'chatbot_model.pt')
print("Modelo guardado como chatbot_model.pt")


Vocabulario: {'hola': 0, 'adiós': 1, 'estás?': 2, 'es': 3, '¿cómo': 4, 'nombre?': 5, '¿qué': 6, 'tu': 7}
Epoch [50/500], Loss: 0.0049
Epoch [100/500], Loss: 0.0016
Epoch [150/500], Loss: 0.0008
Epoch [200/500], Loss: 0.0005
Epoch [250/500], Loss: 0.0003
Epoch [300/500], Loss: 0.0002
Epoch [350/500], Loss: 0.0002
Epoch [400/500], Loss: 0.0001
Epoch [450/500], Loss: 0.0001
Epoch [500/500], Loss: 0.0001
Modelo guardado como chatbot_model.pt


In [2]:
!pip install onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.2 MB/s eta 0:00:00


In [3]:
torch.onnx.export(model,
                  example_input,
                  "chatbot_model.onnx",
                  input_names=['input'],
                  output_names=['output'],
                  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})


/usr/local/lib/python3.10/dist-packages/torch/onnx/symbolic_opset9.py:4279: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

# Datos de ejemplo
data = [
    ("hola", "¡Hola! ¿Cómo estás?"),
    ("¿cómo estás?", "Estoy bien, gracias por preguntar."),
    ("adiós", "¡Adiós! Cuídate."),
    ("¿qué es tu nombre?", "Soy un chatbot simple."),
]

# Preprocesamiento del texto
def tokenize(sentence):
    return sentence.lower().split()

def build_vocab(data):
    words = set()
    for question, answer in data:
        words.update(tokenize(question))
    return {word: idx for idx, word in enumerate(words)}

vocab = build_vocab(data)
vocab_size = len(vocab)

# Convertir texto a tensores
def text_to_tensor(text, vocab):
    tokens = tokenize(text)
    indices = [vocab.get(word, 0) for word in tokens]
    return torch.tensor(indices, dtype=torch.float32)  # Importante: float32

# Red neuronal sin LSTM (para evitar Cast)
class ChatbotModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes):
        super(ChatbotModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.fc1 = nn.Linear(embed_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embedding(x.long())  # Asegurar float32
        x = x.mean(dim=1)  # Promedio de los embeddings
        x = torch.relu(self.fc1(x))
        out = self.fc2(x)
        return out

# Parámetros del modelo
embed_size = 8
hidden_size = 16
num_classes = len(data)
model = ChatbotModel(vocab_size, embed_size, hidden_size, num_classes)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Entrenamiento del modelo
epochs = 500
for epoch in range(epochs):
    total_loss = 0
    for idx, (question, answer) in enumerate(data):
        x = text_to_tensor(question, vocab).unsqueeze(0)
        y = torch.tensor([idx])
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(data):.4f}")

# Exportar modelo a ONNX (opset 11)
example_input = text_to_tensor("hola", vocab).unsqueeze(0)
model = model.float()  # Asegurar float32
torch.onnx.export(
    model,
    example_input,
    "chatbot_model.onnx",
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size', 1: 'sequence_length'}, 'output': {0: 'batch_size'}},
    opset_version=11  # Usar opset 11
)

print("Modelo exportado como chatbot_model.onnx")


Epoch [50/500], Loss: 0.0035
Epoch [100/500], Loss: 0.0009
Epoch [150/500], Loss: 0.0004
Epoch [200/500], Loss: 0.0002
Epoch [250/500], Loss: 0.0002
Epoch [300/500], Loss: 0.0001
Epoch [350/500], Loss: 0.0001
Epoch [400/500], Loss: 0.0001
Epoch [450/500], Loss: 0.0000
Epoch [500/500], Loss: 0.0000
Modelo exportado como chatbot_model.onnx
